In [58]:
import random
import simpy
import numpy as np
import scipy.stats as st
import pandas as pd
import math

In [59]:
RANDOM_SEED = 42
TASK_MEAN = 24.0
# TASK_SIGMA = 2.0

INTERRUPTION_MEAN = 10.0
INTERRUPTION_SIGMA = 3.0

BREAK_MEAN = 6.0


random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [60]:
def time_per_task():
    return st.expon.rvs(size=1, scale=TASK_MEAN)[0]

def time_to_interrupt(enabled=1):
    if enabled:
        return st.expon.rvs(size=1, scale=INTERRUPTION_MEAN)[0]
    return 10000

def time_to_break(fixed=0):
    if fixed:
        return fixed
    return time_per_task()

def break_duration(fixed=0):
    if fixed:
        return fixed
    return st.expon.rvs(size=1, scale=BREAK_MEAN)[0]

def interruption_duration(enabled=1):
    if enabled:
        return st.expon.rvs(size=1, scale=INTERRUPTION_SIGMA)[0]
    return 0



In [61]:



class Person:
    def __init__(self,env,state, b_time, b_duration, interr=True, verbose=False):
        # Simpy Config
        self.state = state
        self.env = env
        self.person = simpy.PreemptiveResource(env,capacity=1)

        #Variables
        self.completed_tasks = 0
        self.breaks = 0
        self.interrupts = 0
        self.task_duration_sum = 0
        self.break_duration_sum = 0

        # Metodología
        self.b_time, self.b_duration = b_time, b_duration
        self.interrupting_enabled = interr
        self.verbose = verbose
        
        #Procesos
        self.process_working = env.process(self.working())
        env.process(self.interrupting())
        self.process_break = env.process(self.take_break())

    def printv(self, s):
        # print(self.verbose,args)
        if self.verbose:
            print(s)

    def working(self):
        while True:
            time = time_per_task()
            t_duration = time
            
            # print('duracion de la proxima tarea', time)
            it = ""
            
            while time:    
                start = self.env.now
                try:
                    self.printv(f'Minuto {self.env.now} | {it}iniciando tarea {self.completed_tasks+1} | queda por trabajar {time} min')
                    self.state = 'W'

                    yield self.env.timeout(time)
                    
                    time = 0
                    self.state = 'S'

                except simpy.Interrupt:
                    time = max(0, time + start - self.env.now)
                    
                    if self.state == 'I':
                        interruption_time = interruption_duration()
                        self.printv(f'Minuto {self.env.now} | trabajo interrumpido por {interruption_time} | quedando por completar unos {time} min de tarea')
                        
                        yield self.env.timeout(interruption_time)
                        self.printv(f'Minuto {self.env.now} | interrupción terminada')
                        self.interrupts += 1
                        #self.state = 'W'
                        
                    elif self.state == 'D': 
                        break_time = break_duration(self.b_duration)
                        break_t = break_time
                        start_break = self.env.now
                        self.printv(f'Minuto {self.env.now} | descanso por {break_time} min | quedando por completar unos {time} min de tarea')

                        while break_time:
                            try:
                                yield self.env.timeout(break_time)
                                break_time = 0
                                #self.state = 'W'

                            except:
                                break_time = self.env.now - start_break
                                interruption_time = interruption_duration()
                                self.printv(f'Minuto {self.env.now} | descanso interrumpido por {interruption_time} min')
                                yield self.env.timeout(interruption_time)
                                self.printv(f'Minuto {self.env.now} | interrupción terminada')
                                self.interrupts += 1
                                # yield self.env.timeout(break_time)
                                self.state = 'D'

                        self.breaks += 1
                        self.break_duration_sum += break_t
                        self.printv(f'Minuto {self.env.now} | descanso completado en el minuto {self.env.now}' )
                                
                            

                    # yield self.env.timeout(time)
                    # time = 0
                    # self.state = 'S'            

                it='re'    
                
            self.completed_tasks += 1
            self.task_duration_sum += t_duration
            self.printv(f'Minuto {self.env.now} | tarea numero {self.completed_tasks} completada')

    def take_break(self):
        dur = 0.0
        while True:
            time = time_to_break(self.b_time + dur)
            dur = self.b_duration
            # print('solicitando el proximo descanso en ',self.env.now, ' para dentro de ', time)
            yield self.env.timeout(time)
           
            with self.person.request(priority=1) as request:
                yield request
                    
                if self.state == 'W':
                    self.state = 'D'
                    self.process_working.interrupt()
                # elif self.state == 'I':
                #     print('no puedes descansar por interrupcion', self.env.now)
                

    def interrupting(self):
        while True:
            time = time_to_interrupt(self.interrupting_enabled)
            # print('solicitando proxima interrupcion en ', self.env.now, ' para dentro de :', time)
            yield self.env.timeout(time)
            
            with self.person.request(priority=0) as request:
                yield request
                # print('se obtuvo la interrupcion (el recurso) en el minuto', self.env.now)
                if self.state == 'W':
                    self.state = 'I'
                    self.process_working.interrupt()
                elif self.state == 'D':
                    self.state = 'I'
                    self.process_working.interrupt()
                else:
                    self.printv(f'no se puede interrumpir en el minuto {self.env.now}')
                    continue
            
    

Exception ignored in: <generator object Person.working at 0x158809e00>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/tokenize.py", line 527, in _tokenize
    pseudomatch = _compile(PseudoToken).match(line, pos)
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Person.working at 0x158809690>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/tokenize.py", line 527, in _tokenize
    pseudomatch = _compile(PseudoToken).match(line, pos)
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Person.working at 0x158926c70>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/tokenize.py", line 527, in _tokenize
    pseudomatch = _compile(PseudoToken).match(line, pos)
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object 

## Preguntas
- Cual es la media de la cantidad de tareas resultas siguiendo cada estrategia?

- Comparar que estrategia de administración de tiempo es mas productiva?
  - 24-6
  - 12-3
  - Free

- ¿Cuál es la distribución de la cantidad de tareas completadas en la ventana de tiempo de 480 minutos? Esto te permitiría entender cuánto varía el rendimiento de la persona.

- ¿Cómo afecta la duración de los descansos a la cantidad de tareas completadas? Podrías comparar las simulaciones con diferentes duraciones de descanso para ver si hay alguna diferencia significativa.

- ¿Cómo afectan las interrupciones a la cantidad de tareas completadas? Similar a la pregunta anterior, podrías comparar las simulaciones con y sin interrupciones.

- ¿Cuál es la distribución del tiempo entre tareas completadas? Esto te permitiría entender cómo se distribuye el tiempo de trabajo de la persona.

- ¿Cómo se correlacionan el número de descansos y el número de tareas completadas? Esto podría ayudarte a entender si tomar más descansos lleva a completar más tareas.

In [64]:
random.seed(RANDOM_SEED)

def simulate(n, time, b_time=0, b_duration=0, interruptions=True, verbose=False, likelihood=False, MSE=10):
    df = pd.DataFrame({})

    i=0
    invalid = likelihood
    while invalid or i < n:
        env = simpy.Environment()
        person = Person(env,'S', b_time, b_duration, interruptions)
        env.run(until=time)

        # Data Collecting
        row = pd.DataFrame(
            {  
                "break_duration": [b_duration], 
                "break_timestamp": [b_time],
                "interruptions": [person.interrupts],
                "completed_tasks": [person.completed_tasks],
                "breaks": [person.breaks],
                "working_time": [person.task_duration_sum],
                "break_time": [person.break_duration_sum],
            }
        )
        df = pd.concat([df,row])

        # To verify stopping criteria
        if likelihood:
            # new_params = np.array([
            #     person.interrupts,
            #     person.completed_tasks, 
            #     person.breaks, 
            #     person.task_duration_sum/(person.completed_tasks + 1 - np.sign(person.completed_tasks)),
            #     person.break_duration_sum/(person.breaks + 1 - np.sign(person.breaks))
            # ])
            # aux = (new_params - est_mean)/(i+1)
            # if i > 0:
            #     est_std = (i-1)/i*est_std + (i+1)*(aux)**2
            # est_mean += aux
            
            est_std = df.std()
            print(est_std/np.sqrt(i+1))
            invalid = len(est_std[est_std/np.sqrt(i+1) < MSE]) < 7
        

        # If the simulation verbosity is true the print the outcomes
        if verbose:
            print("Descanso: ",person.breaks)
            print("Tareas Completadas: ",person.completed_tasks)
            print("Duración Media de tareas: ",person.task_duration_sum/person.completed_tasks)
            print("Interrupciónes: ",person.interrupts)

        i+=1
        
    return df



Exception ignored in: <generator object Person.working at 0x158d1ef80>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Person.working at 0x158d1f060>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Person.working at 0x158d1d850>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Person.working at 0x158d1c190>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Person.working at 0x158d1c660>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Perso

## Extracting Data


In [63]:


# df_rand = simulate(30, 240)
# df_24_6 = simulate(30, 240, 24, 6)
df_12_3 = simulate(30, 240, 12, 3,likelihood=True)

# df_12_3.head()

Exception ignored in: <generator object Person.working at 0x158c23c30>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/internals/managers.py", line 2246, in _stack_arrays
    placement, arrays = zip(*tuples)
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Person.working at 0x158cbe650>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/internals/managers.py", line 2246, in _stack_arrays
    placement, arrays = zip(*tuples)
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Person.working at 0x158cbd5b0>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/internals/managers.py", line 2246, in _stack_arrays
    placement, arrays = zip(*tuples)
RuntimeError: generator

break_duration    NaN
break_timestamp   NaN
interruptions     NaN
completed_tasks   NaN
breaks            NaN
working_time      NaN
break_time        NaN
dtype: float64
break_duration      0.000000
break_timestamp     0.000000
interruptions       2.828427
completed_tasks     0.707107
breaks              3.535534
working_time       16.857562
break_time         10.606602
dtype: float64
break_duration      0.000000
break_timestamp     0.000000
interruptions       3.055050
completed_tasks     2.645751
breaks              3.214550
working_time       12.043426
break_time          9.643651
dtype: float64
break_duration      0.000000
break_timestamp     0.000000
interruptions       2.581989
completed_tasks     2.217356
breaks              2.708013
working_time       10.934973
break_time          8.124038
dtype: float64
break_duration      0.000000
break_timestamp     0.000000
interruptions       3.847077
completed_tasks     2.000000
breaks              2.387467
working_time       12.644210
bre

In [56]:
# import matplotlib.pyplot as plt
# import seaborn as sns


# sns.histplot(data = df_12_3['working_time'], bins=24)
# plt.show()

print(df_12_3['break_time'].max())
print(df_12_3['break_time'].min())

# print(df_12_3[df_12_3['break_time'] == df_12_3['break_time'].max()])
# df_12_3[df_12_3['break_time'] == df_12_3['break_time'].max()]
df_12_3.reset_index()

42
0


,index,break_duration,break_timestamp,interruptions,completed_tasks,breaks,working_time,break_time
0,0,3,12,21,1,3,21.910621,9
1,0,3,12,21,10,14,138.858145,42
2,0,3,12,18,2,13,77.623382,39
3,0,3,12,20,6,12,123.860966,36
4,0,3,12,15,6,9,152.390215,27
5,0,3,12,20,7,10,126.126703,30
6,0,3,12,23,6,10,117.877897,30
7,0,3,12,13,5,13,135.645759,39
8,0,3,12,18,3,11,97.092283,33
9,0,3,12,12,5,11,86.316965,33
